In [11]:
import math
import matplotlib.pyplot as plt
import numpy as np
import pygame
import copy
import time
from math import*

# Function Definitions

In [12]:
# Current X, Y position + ObstacleMap(n,m array)
def checkActions(x, y, obstacleSpace):
#     u, ur, r, dr, d, dl, l, ul
    available = []
#     Up
    if obstacleSpace[x][y+1] == 0:
        available.append([0, 1])
#     Up,Right
    if obstacleSpace[x+1][y+1] == 0:
        available.append([1, 1])
#     Right
    if obstacleSpace[x+1][y] == 0:
        available.append([1, 0])
#     Down, Right
    if obstacleSpace[x+1][y-1] == 0:
        available.append([1, -1])
#     Down
    if obstacleSpace[x][y-1] == 0:
        available.append([0, -1])
#     Down, Left
    if obstacleSpace[x-1][y-1] == 0:
        available.append([-1, -1])
#     Left
    if obstacleSpace[x-1][y] == 0:
        available.append([-1, 0])
#     Up, Left
    if obstacleSpace[x-1][y+1] == 0:
        available.append([-1, 1])
        
    return available


# XY = [X, Y]
# Vn = Visited Nodes (list)
def checkVisited(xy, Vn):
    visited = 0
    for i in range(len(Vn)):
        if Vn[i][0][0] == xy[0]:
            if Vn[i][0][1] == xy[1]:
                visited = 1
    return visited


# Returns list of nodes starting at end node ends at start node
def backTrace(endNode, startNode, Vn):
    nodeToTest = endNode
    nodeList = []
    nodeList.append(endNode)
    while nodeToTest[0] != startNode[0]:
        parent = nodeToTest[2]
        for i in range(len(Vn)):
            if Vn[i][0] == parent:
                nodeList.append(Vn[i])
                nodeToTest = Vn[i]
        
    return nodeList

# create new node structured
def createNode(xy, cost, parent, Vn):
    if checkVisited(xy, Vn) == 0:
        returnList = [[xy[0], xy[1]], cost, parent]
        return returnList
    else:
        pass
    
    
# Sort List from low to high by the given index of the inner list (1 for cost)
def sortList(nodeList, index): 
    for i in range(0, len(nodeList)): 
        for j in range(0, len(nodeList)-i-1): 
            if (nodeList[j][index] > nodeList[j + 1][index]): 
                temp = nodeList[j] 
                nodeList[j]= nodeList[j + 1] 
                nodeList[j + 1]= temp 
    return nodeList 

def checkHalfPlane(X0, Y0, x, y):
    a = (X0[1] - Y0[1])/(X0[0] - Y0[0])
    b = X0[1] - a*X0[0]
    
    check = y - b - a*x
    
    if check <= 0:
        return 1
    return 0

# Check for obstcale (return 1 if obstacle, 0 if not)
def checkObs(x,y):
#     Circle
    circle = (x-225)**2 + (y-150)**2 -(25**2)
    if circle <=0:
        return 1
#     Elipse
    ellipse= ((x-150)/40)**2 + ((y-100)/20)**2 -1
    if ellipse<=0:
            return 1
    
#     Rectangle
    X=[95,30]
    Y=[95-(cos(pi/6)*75), 30+(75*sin(pi/6))]
    Z=[95+(10*sin(pi/6)), 30+(10*cos(pi/6))]
    W=[Z[0]-(cos(pi/6)*75), Z[1]+(75*sin(pi/6))]
    
    if checkHalfPlane(X, Y, x, y)==0 and checkHalfPlane(X, Z, x, y)==0 and checkHalfPlane(Y, W, x, y)==1 and checkHalfPlane(Z, W, x, y)==1:
            return 1

    
#    Rhombus
    A = [225,10]
    B = [250,25]
    C = [225,40]
    D = [200,25]
    
    if checkHalfPlane(A, B, x, y)==0 and checkHalfPlane(B, C, x, y)==1 and checkHalfPlane(C, D, x, y)==1 and checkHalfPlane(A, D, x, y)==0:
            return 1

        
#     Polygon
    P1 = [20, 120]
    P2 = [25, 185]
    P3 = [75, 185]
    P4 = [100, 150]
    P5 = [75, 120]
    P6 = [50, 150]
# Check Triange 1 (P1, P2, P6)
    if checkHalfPlane(P1, P2, x, y)==1 and checkHalfPlane(P2, P6, x, y)==1 and checkHalfPlane(P1, P6, x, y)==0:
            return 1
# Check Triange 2 (P2, P3, P6)
    if checkHalfPlane(P3, P2, x, y)==1 and checkHalfPlane(P2, P6, x, y)==0 and checkHalfPlane(P3, P6, x, y)==0:
            return 1
# Check Triange 3 (P4, P3, P6)
    if checkHalfPlane(P3, P4, x, y)==1 and checkHalfPlane(P4, P6, x, y)==0 and checkHalfPlane(P3, P6, x, y)==1:
            return 1
# Check Triange 3 (P4, P5, P6)
    if checkHalfPlane(P5, P4, x, y)==0 and checkHalfPlane(P4, P6, x, y)==1 and checkHalfPlane(P5, P6, x, y)==0:
            return 1
    
    return 0

# Generate Map (including obstacles)
def generateMap():
    mapList = []
    rowList = []
    for x in range(300):
        for y in range(200):
            if checkObs(x, y) == 1:
                rowList.append(1)
            else:
                rowList.append(0)

        
        mapList.append(rowList)
        rowList = []
        
    return mapList

# Generate Map + Obstacle Clearance (Radius and Clearance of Agent)
def generateObstacleSpace(r, c, obstacleMap):
    circleList = []
    rowList = []
    obstacleSpace = copy.deepcopy(obstacleMap)
    
    for x in range(2*(r+c)+1):
        for y in range(2*(r+c)+1):
            circle = (x-(r+c))**2 + (y-(r+c))**2 - (r+c+0.5)**2
            if circle <= 0:
                rowList.append(1)
            else:
                rowList.append(0)
            
        circleList.append(rowList)
        rowList = []
        
    for row in range(len(obstacleMap)):
        for column in range(len(obstacleMap[row])):
            
            if obstacleMap[row][column] == 1:
                
                for inColumn in range(len(circleList)):
                    for inRow in range(len(circleList)):
                        if row+inRow-(r+c) < len(obstacleMap) and column+inColumn-(r+c) < len(obstacleMap[row+inRow-(r+c)]):
                            if obstacleMap[row+inRow-(r+c)][column+inColumn-(r+c)] == 0 and circleList[inRow][inColumn]:
                                obstacleSpace[row+inRow-(r+c)][column+inColumn-(r+c)] = 2
                            
            if row <= r+c-1:
                if obstacleSpace[row][column] == 0:
                    obstacleSpace[row][column] = 2
                
            if row >= len(obstacleMap)-(r+c):
                if obstacleSpace[row][column] == 0:
                    obstacleSpace[row][column] = 2
                
            if column <= r+c-1:
                if obstacleSpace[row][column] == 0:
                    obstacleSpace[row][column] = 2
                
            if column >= len(obstacleMap[row])-(r+c):
                if obstacleSpace[row][column] == 0:
                    obstacleSpace[row][column] = 2
 
    return obstacleSpace

## Djikstra Algorithm

In [13]:
# Dijkstra Algorithm
def dijkstra(startNode, goalNode, obstacles):
#     Empty Lists
    nodes = []
    nextNodes = []
    Q = []
    Vn = []   
    
#   Q = List of Nodes in Queue
#   Vn = List of visited nodes
    Q.append(startNode)
    Vn.append(startNode)
    
#   Variable to check if there is a new node with a lowest cost    
    newLowerCost = 1;
    run = 0
    
    
    while len(Q) > 0:

#         Sort List if there is a new node with a lower cost than the first node in Q
        if newLowerCost:
            sortList(Q, 1)
            newLowerCost = 0
            
#         X = first first node in Q           
        x = Q.pop(0)
        
#         U = action set at given coordinates of X
        u = checkActions(x[0][0], x[0][1], obstacles)
#         For all actions:
        for i in range(len(u)):

#             Generate X, Y and Cost of new Node
            newX = x[0][0] + u[i][0]
            newY = x[0][1] + u[i][1]
            newCost = x[1]+math.sqrt(abs(u[i][0])+abs(u[i][1]))

#             Check if goal is reached
            if [newX, newY] == goalNode[0]:
                lastNode = createNode([newX, newY], newCost,  x[0], Vn)
                nextNodes.append(lastNode)
                Vn.extend(nextNodes)
#                 Leave function and return the reached node and all visited nodes
                return [lastNode, Vn]
            
    
            else:
                newNode = createNode([newX, newY], newCost,  x[0], Vn)
                 
#                 createNode might return NONE if all nodes have already been visited 
#                 therefore check if actually a new node is created
                if newNode:
                    nextNodes.append(newNode)
#                 Extend the Vn and Q list
                Vn.extend(nextNodes)
                Q.extend(nextNodes)
                nextNodes = [];
 
        if len(Q) > 0:
            if x[1] < Q[0][1]:
                newLowerCost = 1;

        run = run+1
    print('Error, no path found')
    return [Vn[len(Vn)-1], Vn]

## Animation (PyGame, Map)

In [14]:
def drawMap(backList, Vn, start, goal, obstacleSpace, r, showGrid):

    # Define colors
    OBSTACLE_COLOR = (0, 0, 0)
    BACKGROUND_COLOR = (255, 255, 255)
    OBSTACLE_SPACE_COLOR = (150, 150, 255) # Light Blue
    PATH_COLOR = (0, 220, 0)
    GRID_COLOR = (230, 230, 230)
    AGENT_COLOR = (255, 100, 0)
    AGENT_TRACE_COLOR = (255, 200, 90)
    START_END_POINT_COLOR = (255, 0, 255)
    
    
    # This sets the WIDTH and HEIGHT of grid and map
    WIDTH = 3
    HEIGHT = 2
    MAP_WIDTH = 300
    MAP_HEIGHT = 200

    # margin between each cells
    if showGrid:
        MARGIN = 1
    else:
        MARGIN = 0
        
# Create Grid Array
    grid = []
    for row in range(MAP_HEIGHT):
        grid.append([])
        for column in range(MAP_WIDTH):
            grid[row].append([0, 0])

# Initialize pygame
    pygame.init()
# Set the HEIGHT and WIDTH of the screen
    WINDOW_SIZE = [MAP_WIDTH*(WIDTH+MARGIN)-1, MAP_HEIGHT*(HEIGHT+MARGIN)-1]
    screen = pygame.display.set_mode((WINDOW_SIZE))

# Set title of screen
    pygame.display.set_caption("Animation - Dijkstra Algorithm")
# Loop until the user clicks the close button.
    done = False
# Used to manage how fast the screen updates
    clock = pygame.time.Clock()
# Set screen background
    screen.fill(GRID_COLOR)
    
    
# Get Obstacle or ObstacleSpace
    for i in range(MAP_WIDTH):
        for j in range(MAP_HEIGHT):
            if obstacleSpace[i][MAP_HEIGHT-j-1] == 1:
                grid[j][i][0] = 1            
            if obstacleSpace[i][MAP_HEIGHT-j-1] == 2:
                grid[j][i][0] = 2

# Draw Obstacle and ObstacleSpace
    for row in range(MAP_HEIGHT):
        for column in range(MAP_WIDTH):
            color = BACKGROUND_COLOR
            if grid[row][column][0] == 2:
                color = OBSTACLE_SPACE_COLOR 
            if grid[row][column][0] == 1:
                color = OBSTACLE_COLOR
                
            pygame.draw.rect(screen,
                             color,
                             [(MARGIN + WIDTH) * column + MARGIN,
                              (MARGIN + HEIGHT) * row + MARGIN,
                              WIDTH,
                              HEIGHT])
            
#     Some Variables
    run = 0
    pathRun = 0
    delay = 0
    startCircleRun = 0
    circleFrameCount = 0
    circleFrameRate = 0
    
#     Main Loop
    while not done:
        
#         Draw Visited Nodes (Vn) (with gradient colors)
        if run < len(Vn):
            color = (255, abs(255-((Vn[run][1]/backList[0][1])*150)), abs(255-((Vn[run][1]/backList[0][1])*150)))
            pygame.draw.rect(screen,
             color,
             [(MARGIN + WIDTH) * Vn[run][0][0] + MARGIN,
              (MARGIN + HEIGHT) * (MAP_HEIGHT-Vn[run][0][1]-1) + MARGIN,
              WIDTH,
              HEIGHT])
       
        
#         for i in range(len(backList)):
#             grid[MAP_HEIGHT-backList[i][0][1]-1][backList[i][0][0]][0] = 2        
    
# Draw Path 
        if run > len(Vn):
            for i in range(len(backList)):
                pygame.draw.rect(screen,
                 PATH_COLOR,
                 [(MARGIN + WIDTH) * backList[i][0][0] + MARGIN,
                  (MARGIN + HEIGHT) * (MAP_HEIGHT-backList[i][0][1]-1) + MARGIN,
                  WIDTH,
                  HEIGHT])
            pathRun = 1
            
        
#        Draw Agent, following path 
        if pathRun == 1:
        
#         Agent Trace
            if circleFrameCount < len(backList):
                for i in range(circleFrameCount):
                    pygame.draw.circle(screen, 
                       AGENT_TRACE_COLOR, 
                       [round((MARGIN + WIDTH) * backList[len(backList)-i-1][0][0]  + (WIDTH/2) + MARGIN ), 
                       round((MARGIN + HEIGHT) * (MAP_HEIGHT-backList[len(backList)-i-1][0][1]) - (HEIGHT/2))],
                       r*(WIDTH+MARGIN))
                    
#                 Agent
                pygame.draw.circle(screen, 
                   AGENT_COLOR, 
                   [round((MARGIN + WIDTH) * backList[len(backList)-circleFrameCount-1][0][0]  + (WIDTH/2) + MARGIN ), 
                   round((MARGIN + HEIGHT) * (MAP_HEIGHT-backList[len(backList)-circleFrameCount-1][0][1]) - (HEIGHT/2))],
                   r*(WIDTH+MARGIN)-1)
    
#                 Agent Border
                pygame.draw.circle(screen, 
                   (0, 0, 0), 
                   [round((MARGIN + WIDTH) * backList[len(backList)-circleFrameCount-1][0][0]  + (WIDTH/2) + MARGIN ), 
                   round((MARGIN + HEIGHT) * (MAP_HEIGHT-backList[len(backList)-circleFrameCount-1][0][1]) - (HEIGHT/2))],
                   r*(WIDTH+MARGIN)-1, 
                    1)
#                 Redraw Path (over agent + trace)
                for i in range(len(backList)):
                    pygame.draw.rect(screen,
                     PATH_COLOR,
                     [(MARGIN + WIDTH) * backList[i][0][0] + MARGIN,
                      (MARGIN + HEIGHT) * (MAP_HEIGHT-backList[i][0][1]-1) + MARGIN,
                      WIDTH,
                      HEIGHT])
                
# Slow down agent movement     
                if circleFrameRate > 50:
                    circleFrameCount = circleFrameCount + 1
                    circleFrameRate = 0
                
                circleFrameRate = circleFrameRate + 1         
                
#       Draw Start- and Goal Point
        pygame.draw.rect(screen,
         START_END_POINT_COLOR,
         [(MARGIN + WIDTH) * start[0][0] + MARGIN,
          (MARGIN + HEIGHT) * (MAP_HEIGHT-start[0][1]-1) + MARGIN,
          WIDTH,
          HEIGHT])
        pygame.draw.rect(screen,
         START_END_POINT_COLOR,
         [(MARGIN + WIDTH) * goal[0][0] + MARGIN,
          (MARGIN + HEIGHT) * (MAP_HEIGHT-goal[0][1]-1) + MARGIN,
          WIDTH,
          HEIGHT])
        

        run = run+1
         
        for event in pygame.event.get():  # User did something
            if event.type == pygame.QUIT:  # If user clicked close
                done = True  # Flag that we are done so we exit this loop



        # Limit to 1000 frames per second
        clock.tick(1000)

        # update the screen
        pygame.display.flip()


    print('Program Ended')
    pygame.quit()  

# Main Script

In [15]:
# Node Structure: [[x, y], cost, [parentX, parentY]]
start_time = time.time()
#  Set Start and Goal Position (X, Y)
#    with (X, Y) = (0, 0) being the bottom left corner
startNode = [[241, 17]]
goalNode = [[200, 10]]
# startNode = [[241, 17]]
# goalNode = [[80, 36]]

# Radius and Clearance for Rigid Robot
r = 2
c = 0

# Show Grid on Map
showGrid = True


startNode = [[5, 5]]
goalNode = [[295, 195]]

# Generate Map and Obstacle Space
obstacles = generateMap()
obstacleSpace = generateObstacleSpace(r, c, obstacles)


# Check if Start and End point are in free space
if obstacleSpace[startNode[0][0]][startNode[0][1]] != 0:
    raise Exception("Start lies inside obstacle space")
if obstacleSpace[goalNode[0][0]][goalNode[0][1]] != 0:
    raise Exception("Goal lies inside obstacle space")


# Structure Start Node correctlz
startNode.append(0)
startNode.append([0,0])



# Run Djikstra Algorithm
[reachedNode, Vn] = dijkstra(startNode, goalNode, obstacleSpace)
# Generate Path (From Goal to Start, following parents)
backList = backTrace(reachedNode, startNode, Vn)
end_time=time.time()
print(end_time-start_time)
delta = (end_time-start_time)/60
print(delta)
# Create Animation
drawMap(backList, Vn, startNode, goalNode, obstacleSpace, r, showGrid)


1729.0849404335022
28.81808234055837
Program Ended
